<a href="https://colab.research.google.com/github/DreRnc/ExplainingExplanations/blob/main/Explanations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset : **E-SNLI**. \
Model : **Small T5**.

In [1]:
colab = False

In [2]:
if colab:
    !git clone https://github.com/DreRnc/ExplainingExplanations.git
    %cd ExplainingExplanations
    %pip install -r requirements.txt

# 1.0 Preparation


## 1.1 Loading Dataset

In [3]:
from datasets import load_dataset

dataset = load_dataset("esnli")

/Users/dre/Desktop/GitRepo/DreRnc/ExplainingExplanations/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
training_set = dataset["train"]
validation_set = dataset["validation"]
test_set = dataset["test"]

print("Shape of training_set: ", training_set.shape)
print("Shae of validation_set: ", validation_set.shape)
print("Shape of test_set: ", test_set.shape)

Shape of training_set:  (549367, 6)
Shae of validation_set:  (9842, 6)
Shape of test_set:  (9824, 6)


In [5]:
training_set[0]

{'premise': 'A person on a horse jumps over a broken down airplane.',
 'hypothesis': 'A person is training his horse for a competition.',
 'label': 1,
 'explanation_1': 'the person is not necessarily training his horse',
 'explanation_2': '',
 'explanation_3': ''}

In [6]:
n_train = n_valid = n_test = 50

train_small = training_set.select(range(n_train))
valid_small = validation_set.select(range(n_valid))
test_small = test_set.select(range(n_test))

print("Shape of train_small: ", train_small.shape)
print("Shape of valid_small: ", valid_small.shape)
print("Shape of test_small: ", test_small.shape)

Shape of train_small:  (50, 6)
Shape of valid_small:  (50, 6)
Shape of test_small:  (50, 6)


## 1.2 Loading T5 Model

In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Test **zero-shot** on a random task.

In [8]:
input_ids = tokenizer(
    "translate English to French: Hello Dre, I think the English version is ok for us.",
    return_tensors="pt",
).input_ids
outputs = model.generate(input_ids, max_new_tokens=100)

print(tokenizer.decode(outputs[0], skip_special_tokens=True, max_length=100))

torch.Size([1, 21])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
Bonjour Dre, je pense que la version anglaise est bonne pour nous.


## 1.3 Zero-shot example to Verify Everything is Working

In [9]:
from src.utils import generate_prompt_mnli

In [10]:
example = training_set[0]
example

{'premise': 'A person on a horse jumps over a broken down airplane.',
 'hypothesis': 'A person is training his horse for a competition.',
 'label': 1,
 'explanation_1': 'the person is not necessarily training his horse',
 'explanation_2': '',
 'explanation_3': ''}

Generating the prompt:

<b><u> mnli hypothesis: </b></u> The St. Louis Cardinals have always won. <b><u> premise: </b></u> yeah well losing is i mean i’m i’m originally from Saint Louis and Saint Louis Cardinals when they were there were uh a mostly a losing team but

Output:
* 0: Entailment
* 1: Neutral
* 2: Contradiction

In [11]:
prompt = generate_prompt_mnli(example)
prompt

'mnli hypothesis: A person is training his horse for a competition. premise: A person on a horse jumps over a broken down airplane.'

In [12]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(outputs)
print('Shape of outputs:', outputs.shape)
print('Shape of outputs[0]:', outputs[0].shape)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

torch.Size([1, 37])
torch.Size([1, 1])
torch.Size([1, 1])
tensor([[   0, 7163,    1]])
Shape of outputs: torch.Size([1, 3])
Shape of outputs[0]: torch.Size([3])
neutral


/Users/dre/Desktop/GitRepo/DreRnc/ExplainingExplanations/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


## 1.4  Tokenize the dataset

In [13]:
train_small.info

DatasetInfo(description='', citation='', homepage='', license='', features={'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None), 'explanation_1': Value(dtype='string', id=None), 'explanation_2': Value(dtype='string', id=None), 'explanation_3': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name='parquet', dataset_name='esnli', config_name='plain_text', version=0.0.2, splits={'train': SplitInfo(name='train', num_bytes=107611996, num_examples=549367, shard_lengths=None, dataset_name='esnli'), 'validation': SplitInfo(name='validation', num_bytes=3416319, num_examples=9842, shard_lengths=None, dataset_name='esnli'), 'test': SplitInfo(name='test', num_bytes=3379781, num_examples=9824, shard_lengths=None, dataset_name='esnli')}, download_checksums={'hf://datasets/esnli@f01d1430c0ad65a3785d06079a0e01e015e769b8/plain_text/

In [14]:
train_small.features

{'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None),
 'explanation_1': Value(dtype='string', id=None),
 'explanation_2': Value(dtype='string', id=None),
 'explanation_3': Value(dtype='string', id=None)}

In [15]:
from functools import partial
from src.utils import tokenize_function
import time

In [16]:
tokenize_mapping = partial(tokenize_function, tokenizer=tokenizer)

In [17]:
time_init = time.time()
train_small_tokenized = train_small.map(tokenize_mapping, batched=True).with_format("torch")
valid_small_tokenized = valid_small.map(tokenize_mapping, batched=True).with_format("torch")
test_small_tokenized = test_small.map(tokenize_mapping, batched=True).with_format("torch")
time_end = time.time()

print("Time taken to tokenize: ", time_end - time_init)
print("Shape of train_small_tokenized: ", train_small_tokenized.shape)
print("Shape of valid_small_tokenized: ", valid_small_tokenized.shape)
print("Shape of test_small_tokenized: ", test_small_tokenized.shape)

Time taken to tokenize:  0.07505202293395996
Shape of train_small_tokenized:  (50, 8)
Shape of valid_small_tokenized:  (50, 8)
Shape of test_small_tokenized:  (50, 8)


In [18]:
train_small_tokenized.features

{'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None),
 'explanation_1': Value(dtype='string', id=None),
 'explanation_2': Value(dtype='string', id=None),
 'explanation_3': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [19]:
print(train_small_tokenized[:4])

{'premise': ['A person on a horse jumps over a broken down airplane.', 'A person on a horse jumps over a broken down airplane.', 'A person on a horse jumps over a broken down airplane.', 'Children smiling and waving at camera'], 'hypothesis': ['A person is training his horse for a competition.', 'A person is at a diner, ordering an omelette.', 'A person is outdoors, on a horse.', 'They are smiling at their parents'], 'label': tensor([1, 2, 0, 1]), 'explanation_1': ['the person is not necessarily training his horse', 'One cannot be on a jumping horse cannot be a diner ordering food.', 'a broken down airplane is outdoors', 'Just because they are smiling and waving at a camera does not imply their parents or anyone is anyone behind it'], 'explanation_2': ['', '', '', ''], 'explanation_3': ['', '', '', ''], 'input_ids': tensor([[    3,    51,    29,    40,    23, 22455,    10,    71,   568,    19,
           761,   112,  4952,    21,     3,     9,  2259,     5,     3, 17398,
            10

# 2.0 Task 1: Zero-shot evaluation

In [20]:
from src.utils import evaluate_output_mnli
from tqdm import tqdm
import torch

In [21]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model.to(device)
device

device(type='cpu')

In [22]:
correct = 0
seen = 0
batch_size = 16 # Choose an appropriate batch size

for i in tqdm(range(0, len(train_small_tokenized), batch_size)):
    batch_dict = train_small_tokenized[i : i + batch_size]

    # Prepare input_ids tensor for the entire batch
    input_ids = batch_dict["input_ids"].to(device)

    # Generate outputs for the batch
    outputs = model.generate(input_ids)

    # Iterate through the batch and evaluate each output
    for idx, output_ids in enumerate(outputs):
        output = tokenizer.decode(output_ids, skip_special_tokens=True)
        correct += evaluate_output_mnli(output, batch_dict["label"][idx])
        seen += 1

print("Accuracy zero-shot on the test set: ", correct / seen)

  0%|          | 0/4 [00:00<?, ?it/s]

torch.Size([16, 128])
torch.Size([16, 1])
torch.Size([16, 1])
torch.Size([16, 1])


 25%|██▌       | 1/4 [00:00<00:02,  1.29it/s]

torch.Size([16, 1])
torch.Size([16, 1])
torch.Size([16, 128])
torch.Size([16, 1])
torch.Size([16, 1])
torch.Size([16, 1])
torch.Size([16, 1])
torch.Size([16, 1])


 50%|█████     | 2/4 [00:01<00:01,  1.45it/s]

torch.Size([16, 128])
torch.Size([16, 1])
torch.Size([16, 1])
torch.Size([16, 1])
torch.Size([16, 1])
torch.Size([16, 1])


100%|██████████| 4/4 [00:02<00:00,  1.87it/s]

torch.Size([2, 128])
torch.Size([2, 1])
torch.Size([2, 1])
Accuracy zero-shot on the test set:  tensor(0.8600)


# 3.0 Task 2: Fine tuning without explanations

In [23]:
import numpy as np
import evaluate
from src.utils import compute_metrics, preprocess_logits_argmax, eval_pred_transform_accuracy
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, T5ForConditionalGeneration

In [24]:
transform_accuracy = partial(eval_pred_transform_accuracy, tokenizer = tokenizer)
compute_accuracy = partial(compute_metrics, pred_transform=transform_accuracy, metric = evaluate.load('accuracy'))

In [25]:
model_ft = T5ForConditionalGeneration.from_pretrained("t5-small")

In [26]:
training_args = Seq2SeqTrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    num_train_epochs=5,
    predict_with_generate=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16
)

In [27]:
trainer = Seq2SeqTrainer(
    model=model_ft,
    args=training_args,
    train_dataset=train_small_tokenized,
    eval_dataset=valid_small_tokenized,
    compute_metrics=compute_accuracy,
    preprocess_logits_for_metrics = preprocess_logits_argmax
)

In [28]:
trainer.train()

  0%|          | 0/20 [00:00<?, ?it/s]

torch.Size([16, 128])
torch.Size([16])


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
trainer.evaluate(test_small_tokenized)

torch.Size([16, 128])
tensor([[ 3, 51, 29,  ...,  0,  0,  0],
        [ 3, 51, 29,  ...,  0,  0,  0],
        [ 3, 51, 29,  ...,  0,  0,  0],
        ...,
        [ 3, 51, 29,  ...,  0,  0,  0],
        [ 3, 51, 29,  ...,  0,  0,  0],
        [ 3, 51, 29,  ...,  0,  0,  0]], device='mps:0')
torch.Size([16, 1])
tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]], device='mps:0')
torch.Size([16, 1])
tensor([[27252],
        [    3],
        [27252],
        [ 7163],
        [ 7163],
        [ 7163],
        [    3],
        [ 7163],
        [27252],
        [ 7163],
        [    3],
        [ 7163],
        [    3],
        [    3],
        [27252],
        [27252]], device='mps:0')
torch.Size([16, 1])
tensor([[ 1],
        [35],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [35],
        [ 1],
        [ 1],
        

ValueError: not enough values to unpack (expected 2, got 1)

# 4.0 Task 4: Making the model generate explanations